# Electrical Energy Billing

In [1]:
import pandas as pd

tab3 = pd.read_csv("tab3_2025.csv", sep=";", decimal=',') # fattori di perdita

tab_oneri_di_rete = pd.read_csv("tab4_2025.csv", sep=";", decimal=',') # tariffe di rete
comp_s1 = tab_oneri_di_rete['componente s1']
comp_s2 = tab_oneri_di_rete['componente s2']  
comp_s3 = tab_oneri_di_rete['componente s3']

tab_class0 = pd.read_csv("Classe0_minmax.csv", sep=";", decimal=',')
tab_ASOS1 = pd.read_csv("ASOS1_minmax.csv", sep=";", decimal=',')
tab_ASOS2 = pd.read_csv("ASOS2_minmax.csv", sep=";", decimal=',')
tab_ASOS3 = pd.read_csv("ASOS3_minmax.csv", sep=";", decimal=',')
tab_VAL= pd.read_csv("VAL_minmax.csv", sep=";", decimal=',')
# print(tab3.head())
# print(tab4.head())


## input:
- type of consumer
- power of contract (kW)
- consumption (kWh)


In [ ]:
# Mapping tra ConsumerType e "Tipologie di contratto" in tab3
def get_consumer_contracted(tab3, consumer_type, contracted_power_kw: float, voltage_kv: float=None):

    df = tab3[tab3["Tipo di Consumatore"] == consumer_type]
    if voltage_kv is not None:
        df = df[
            (df["Tensione Min di Fornitura (kV)"] <= voltage_kv) &
            (df["Tensione Max di Fornitura (kV)"] >= voltage_kv)
        ]
    else:
        df = df[
            (df["Potenza Min di Contratto (kW)"] <= contracted_power_kw) &
            (df["Potenza Max di Contratto (kW)"] >= contracted_power_kw)
        ]

    if df.empty:
        raise ValueError("Nessuna tariffa trovata per i parametri forniti.")
   
    return df.iloc[0]


In [ ]:
# Mapping tra ConsumerType e "Tipologie di contratto" in oneri
import pandas as pd
from common.enums import ConsumerType


def get_oneri_di_sistema(consumer_type, contracted_power_kw: float, voltage_kv: float=None):
    if consumer_type == 0 and contracted_power_kw <= 3: # BT domestico residente
        tab = tab_class0
    elif consumer_type == 1 and contracted_power_kw <= 3: # BT domestico non residente
        tab = tab_ASOS1
    elif consumer_type in [2, 3]: # BT non domestiche, illuminazione pubblica, ricarica EV
        tab = tab_ASOS2
    elif consumer_type == 4 and 0 <= contracted_power_kw <= 10: 
        tab = tab_ASOS2
    elif consumer_type == 5 and contracted_power_kw <= 100:
        tab = tab_ASOS2
    elif consumer_type in [6,7] and contracted_power_kw > 500: # Media/Alta tensione
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv < 380:
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv >= 380: # Altissima tensione >=380 kV
        tab = tab_VAL
    else:
        raise ValueError(f"Tipo di consumatore {consumer_type} non valido o intervallo di potenza non previsto.")
    return tab

In [ ]:
def calculate_electricity_bill_system_costs(
    consumption_kwh,  
    contracted_power_kw: float,
    consumer_type: int,
    tab3: pd.DataFrame,
    comp_s1, comp_s2, comp_s3,
    voltage_kv: float = None
):
    consumer_enum = ConsumerType(consumer_type)
    per_unit_costs, per_kw_costs, per_kwh_costs = 0, 0, 0
    
    if consumer_enum in [0, 1]:
        per_unit_costs += comp_s1 / 100
        per_kw_costs += (comp_s2 / 100)* contracted_power_kw
        per_kwh_costs += (comp_s3 / 100) * consumption_kwh
    else: # NO_DOMESTIC
        row = get_consumer_contracted(tab3, consumer_enum, contracted_power_kw, voltage_kv)
        per_unit_costs += (row['Quota fissa'] / 100) if not pd.isna(row['Quota fissa']) else 0
        per_kw_costs += (row['Quota potenza'] / 100) * contracted_power_kw if not pd.isna(row['Quota potenza']) else 0
        per_kwh_costs += (row['Quota energia'] / 100) * consumption_kwh if not pd.isna(row['Quota energia']) else 0
    
    row_oneri_sistema = get_oneri_di_sistema(consumer_enum, contracted_power_kw, voltage_kv)
    per_unit_costs += (row_oneri_sistema['Quota fissa'] / 100) if not pd.isna(row_oneri_sistema['Quota fissa']) else 0
    per_kw_costs += (row_oneri_sistema['Quota potenza'] / 100) * contracted_power_kw if not pd.isna(row_oneri_sistema['Quota potenza']) else 0
    per_kwh_costs += (row_oneri_sistema['Quota energia'] / 100) * consumption_kwh if not pd.isna(row_oneri_sistema['Quota energia']) else 0

    return per_unit_costs, per_kw_costs, per_kwh_costs


# Using the class ElectricityBillSystemCosts

In [ ]:
def get_quote_oneri_di_sistema(consumer_type, contracted_power_kw: float=None, voltage_kv: float=None):
    if consumer_type == 0 and contracted_power_kw <= 3: # BT domestico residente
        tab = tab_class0
    elif consumer_type == 1 and contracted_power_kw <= 3: # BT domestico non residente
        tab = tab_ASOS1
    elif consumer_type in [2, 3]: # BT non domestiche, illuminazione pubblica, ricarica EV
        tab = tab_ASOS2
    elif consumer_type == 4 and 0 <= contracted_power_kw <= 10: 
        tab = tab_ASOS2
    elif consumer_type == 5 and contracted_power_kw <= 100:
        tab = tab_ASOS2
    elif consumer_type in [6,7] and contracted_power_kw > 500: # Media/Alta tensione
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv < 380:
        tab = tab_ASOS3
    elif consumer_type == 8 and voltage_kv is not None and voltage_kv >= 380: # Altissima tensione >=380 kV
        tab = tab_VAL
    else:
        raise ValueError(f"Tipo di consumatore {consumer_type} non valido o intervallo di potenza non previsto.")
    
    try:
        df = tab[tab["Tipo di Consumatore"] == consumer_type]
        if voltage_kv is not None:
            df = df[
                (df["Tensione Min di Fornitura (kV)"] <= voltage_kv) &
                (df["Tensione Max di Fornitura (kV)"] >= voltage_kv)
            ]
        else:
            df = df[
                (df["Potenza Min di Contratto (kW)"] <= contracted_power_kw) &
                (df["Potenza Max di Contratto (kW)"] >= contracted_power_kw)
            ]
            
        quota_fissa = df['Quota fissa']
        quota_potenza = df['Quota potenza']
        quota_energia = df['Quota energia']
    except KeyError as e:
        raise KeyError(f"Tabella mancante del campo richiesto: {e}")
    return quota_fissa, quota_potenza, quota_energia

quota_fissa, quota_potenza, quota_energia = get_quote_oneri_di_sistema(consumer_type=8, contracted_power_kw=None, voltage_kv=370)
quota_fissa, quota_potenza, quota_energia

(15    292781.16
 Name: Quota fissa, dtype: float64,
 15    361.44
 Name: Quota potenza, dtype: float64,
 15    1.0374
 Name: Quota energia, dtype: float64)

# Try real bill

In [2]:
consumer_type = 0
consumption_kwh = 137  
losses_kwh = 14 
contracted_power_kw = 3.0

energy_unit_price_kWh = 0.134  # €/kWh
marketing_cost_per_month = 8.00  # €/mese
iva_rate = 0.1 

dispbt_cost_per_month = 0.102592  # €/mese
dispatching_price_kWh = 0.009800  # €/kWh
capacity_market_price = 0.004275  # €/kWh

# Tariffe rete --> sigma in cent/kWh
network_fixed = 190  
network_power = 209 
network_energy = 1.189 
# Oneri di sistema --> in cent/kWh
per_unit_costs_class_0 = 0  
per_kw_costs_class_0 = 0    
asos_energy = 2.9677 
arim_energy = 0.1645  
per_kwh_costs_class_0 = arim_energy + asos_energy # cent/kWh

uc3_energy = 0.001560  # €/kWh = Copertura degli squilibri di sistema per distribuzione
uc6_power = 0.016567  # €/kW = Remunerazione dei miglioramenti della continuità (fisso)
uc6_energy = 0.000070  # €/kWh = Remunerazione dei miglioramenti della continuità (variabile)


In [3]:
from entities.electricity_bill_system_costs import ElectricityBillSystemCosts

my_bill = ElectricityBillSystemCosts(
    consumer_type=0,  # Domestico Residente
    consumption_kwh=consumption_kwh,
    contracted_power_kw=contracted_power_kw,
    sigma_1=network_fixed,
    sigma_2=network_power,
    sigma_3=network_energy,
    per_unit_costs_class_0=per_unit_costs_class_0,
    per_kw_costs_class_0=per_kw_costs_class_0,
    per_kwh_costs_class_0=per_kwh_costs_class_0
)
my_oneri_sistema = my_bill.calculate_electricity_bill_system_costs(
    consumer_type=consumer_type, consumption_kwh=consumption_kwh, contracted_power_kw=contracted_power_kw, 
    uc3_energy_euro_per_kWh=uc3_energy, uc6_power_euro_per_kWh=uc6_power, uc6_energy_euro_per_kWh=uc6_energy
)
my_oneri_sistema

(1.9, 6.319700999999999, 6.1433539999999995)

In [4]:
def calculate_energy_cost(
    consumption_kwh,
    losses_kwh,
    energy_unit_price_kWh,
    marketing_cost_per_month,
    dispbt_cost_per_month,
    dispatching_price_kWh,
    capacity_market_price
):
    '''Calcola il costo della materia energia'''
    energy_cost = consumption_kwh * energy_unit_price_kWh
    losses_cost = losses_kwh * energy_unit_price_kWh
    
    total_energy_kwh = consumption_kwh + losses_kwh
    dispatching_cost = total_energy_kwh * dispatching_price_kWh
    capacity_cost = total_energy_kwh * capacity_market_price
    
    total_energy_cost = energy_cost + losses_cost + dispatching_cost + capacity_cost + dispbt_cost_per_month + marketing_cost_per_month
    return total_energy_cost

def calculate_total_bill(total_energy_cost, system_charges, iva: float = 0.1):
    '''Calcola la bolletta compresa di IVA'''
    imponibile = total_energy_cost + sum(system_charges)
    return imponibile * (1 + iva)

In [5]:
energy_cost = calculate_energy_cost(
    consumption_kwh=consumption_kwh,
    losses_kwh=losses_kwh,
    energy_unit_price_kWh=energy_unit_price_kWh,
    marketing_cost_per_month=marketing_cost_per_month,
    dispbt_cost_per_month=dispbt_cost_per_month,
    dispatching_price_kWh=dispatching_price_kWh,
    capacity_market_price=capacity_market_price
)
energy_cost

30.461917000000003

In [8]:
canone_rai = 9.65
total_bill_to_pay = calculate_total_bill(energy_cost, my_oneri_sistema, iva=iva_rate)
print(f"Bill: {round(total_bill_to_pay + canone_rai, 2)}")

Bill: 58.96
